## Scraping song lyrics from the Genius.com ##

This notebook contains the Python code described on my blog in the [scraping genius lyrics post](http://www.johnwmillr.com/blog/2017/scraping-genius-lyrics).

Head over to my [GitHub repository](https://github.com/johnwmillr/geniusapi) to clone my Python wrapper.


## Genius API ##

In [1]:
# Sign up for a free account at Genius.com to access the API
# http://genius.com/api-clients
client_access_token = 'aPa0OLqvXfY8ht00NeneEBzZyvCvVcCiy40c4U-n3nzjZbfLrhEd2trP0hP_1_X3'

In [2]:
# Let's take a look at how we might search for an artist using the Genius API.
import requests
import urllib.request

# Format a request URL for the Genius API
search_term = 'Andy Shauf'
_URL_API = "https://api.genius.com/"
_URL_SEARCH = "search?q="
querystring = _URL_API + _URL_SEARCH + urllib.request.quote(search_term)
request = urllib.request.Request(querystring)
request.add_header("Authorization", "Bearer " + client_access_token)
# request.add_header("User-Agent","curl/7.9.8 (i686-pc-linux-gnu) libcurl 7.9.8 (OpnSSL 0.9.6b) (ipv6 enabled)")
request.add_header("User-Agent", "")

In [3]:
# Now that we’ve formatted the URL, we can make a request to the database.
import json
response = urllib.request.urlopen(request, timeout=3)
raw = response.read()
json_obj = json.loads(raw)

In [4]:
# The JSON object is just a normal python dictionary
json_obj.keys()

dict_keys(['meta', 'response'])

In [5]:
# The 'hits` key stores info on each song in the search result.
# From here it's easy to grab the song title, album, etc.

# List each key contained within a single search hit
[key for key in json_obj['response']['hits'][0]['result']]

['annotation_count',
 'api_path',
 'full_title',
 'header_image_thumbnail_url',
 'header_image_url',
 'id',
 'lyrics_owner_id',
 'lyrics_state',
 'path',
 'pyongs_count',
 'song_art_image_thumbnail_url',
 'stats',
 'title',
 'title_with_featured',
 'url',
 'primary_artist']

In [6]:
# View the song name for each search hit
[song['result']['title'] for song in json_obj['response']['hits']]

['The Magician',
 'Quite Like You',
 'Early to the Party',
 'To You',
 'Wendell Walker',
 'Martha Sways',
 'Twist Your Ankle',
 'The Worst in You',
 "You're Out Wasting",
 'Eyes of Them All']

In [7]:
# URL to artist image
print(json_obj['response']['hits'][0]['result']['primary_artist']['image_url'])

https://images.genius.com/16423bad48ffd400aac3ba86d5b86ed4.850x850x1.jpg


<img src="https://images.genius.com/16423bad48ffd400aac3ba86d5b86ed4.850x850x1.jpg" style="width: 200px;"/> 

### Access a song or artist directly by ID ###

In [8]:
# If you have an artist or song ID, you can access that entry 
# directly by reformatting the request URL.
song_id = 2299297
querystring = "https://api.genius.com/songs/" + str(song_id)
request = urllib.request.Request(querystring)
request.add_header("Authorization", "Bearer " + client_access_token)
request.add_header("User-Agent", "")
response = urllib.request.urlopen(request, timeout=3)
raw = response.read()
json_obj = json.loads(raw)
print(json_obj)
print((json_obj['response']['song']['title'],\
       json_obj['response']['song']['primary_artist']['name']))

{'meta': {'status': 200}, 'response': {'song': {'annotation_count': 1, 'api_path': '/songs/2299297', 'description': {'dom': {'tag': 'root', 'children': [{'tag': 'p', 'children': ['?']}]}}, 'embed_content': "<div id='rg_embed_link_2299297' class='rg_embed_link' data-song-id='2299297'>Read <a href='https://genius.com/The-young-wild-not-a-one-lyrics'>“Not a One” by The\xa0Young Wild</a> on Genius</div> <script crossorigin src='//genius.com/songs/2299297/embed.js'></script>", 'featured_video': True, 'full_title': 'Not a One by\xa0The\xa0Young Wild', 'header_image_thumbnail_url': 'https://images.rapgenius.com/0fcb3103057c4d76c158eb778fa1d935.300x300x1.jpg', 'header_image_url': 'https://images.rapgenius.com/0fcb3103057c4d76c158eb778fa1d935.1000x1000x1.jpg', 'id': 2299297, 'lyrics_owner_id': 93685, 'lyrics_state': 'complete', 'path': '/The-young-wild-not-a-one-lyrics', 'pyongs_count': 3, 'recording_location': None, 'release_date': '2016-08-26', 'song_art_image_thumbnail_url': 'https://images.

In [9]:
import random
import numpy as np

# If you have an artist or song ID, you can access that entry 
# directly by reformatting the request URL.

def findMeanPageViews(numSongs):
    songViews = np.empty((0,))
    print(songViews.dtype)
    for i in range(numSongs):
        song_id = random.randint(0,2299297)
        querystring = "https://api.genius.com/songs/" + str(song_id)
        request = urllib.request.Request(querystring)
        request.add_header("Authorization", "Bearer " + client_access_token)
        request.add_header("User-Agent", "")
        try:
            response = urllib.request.urlopen(request, timeout=3)
            raw = response.read()
            json_obj = json.loads(raw)
            if 'pageviews' in json_obj['response']['song']['stats'].keys() and json_obj['response']['song']['stats']['pageviews'] != 0:
                print(json_obj['response']['song']['stats']['pageviews'])
                songViews = np.append(songViews, json_obj['response']['song']['stats']['pageviews'])
        except:
            pass
    return songViews

pageViews = findMeanPageViews(100)

print(pageViews)
print(np.mean(pageViews))
print(len(pageViews))
       # print((json_obj['response']['song']['title'],\
       #        json_obj['response']['song']['primary_artist']['name']))

float64
12967
8713
5143
26841
[12967.  8713.  5143. 26841.]
13416.0
4


In [10]:
#GetLyrics
from bs4 import BeautifulSoup
import re
def getLyrics(url):
    page = requests.get(url)    
    html = BeautifulSoup(page.text, "html.parser") # Extract the page's HTML as a string

    # Scrape the song lyrics from the HTML
    lyrics = html.find("div", class_="lyrics").get_text().encode('ascii','ignore')
    #lyrics = re.sub('\[.*\]','',lyrics) # Remove [Verse] and [Bridge] stuff
    #lyrics = re.sub('\n{2}','',lyrics)  # Remove gaps between verses        
    #lyrics = str(lyrics).strip('\n')
    return lyrics

In [14]:
def getSongData(start_id,end_id):
    songData = {}
    for i in range(start_id,end_id+1):
        song_id = i
        if (i % 5) == 0:
            print(i)
        querystring = "https://api.genius.com/songs/" + str(song_id)
        request = urllib.request.Request(querystring)
        request.add_header("Authorization", "Bearer " + client_access_token)
        request.add_header("User-Agent", "")
        try:
            response = urllib.request.urlopen(request, timeout=3)
            raw = response.read()
            json_obj = json.loads(raw)
            if 'pageviews' in json_obj['response']['song']['stats'].keys() and json_obj['response']['song']['stats']['pageviews'] != 0:
                #print(json_obj['response']['song']['stats']['pageviews'])
                song = json_obj['response']['song']
                title = song['title']
                song_id = song['id']
                artist = song['album']['artist']['name']
                release_date = song['release_date']
                lyrics = str(getLyrics(song['url']))
                url = song['url']
                pageviews = song['stats']['pageviews']
                currsongdata = {'url':url,'title':title,'artist':artist,'release_date':release_date,'lyrics':lyrics,'page_views':pageviews}
                songData[song_id] = currsongdata
        except Exception as e:
            print(e)
    return songData

def savejson(data):
    with open('songData.json','r+') as outfile:
        loadedData = json.load(outfile)
        toOutput = {}
        toOutput.update(data)
        toOutput.update(loadedData)
        outfile.truncate(0)
        outfile.seek(0,0)
        json.dump(toOutput,outfile)
        
mydata = getSongData(1010001, 1050000)
savejson(mydata)

1010005
1010010
1010015
1010020
1010025
1010030
1010035
HTTP Error 404: Not Found
1010040
1010045
1010050
1010055
1010060
1010065
1010070
1010075
1010080
1010085
1010090
1010095
1010100
1010105
1010110
1010115
1010120
1010125
1010130
1010135
1010140
1010145
1010150
1010155
HTTP Error 404: Not Found
1010160
1010165
1010170
1010175
1010180
1010185
1010190
1010195
1010200
1010205
HTTP Error 404: Not Found
1010210
1010215
1010220
1010225
1010230
1010235
1010240
HTTP Error 404: Not Found
1010245
HTTP Error 404: Not Found
1010250
1010255
1010260
1010265
1010270
1010275
1010280
1010285
1010290
1010295
1010300
1010305
1010310
1010315
1010320
1010325
1010330
HTTP Error 404: Not Found
1010335
1010340
1010345
1010350
1010355
1010360
1010365
1010370
HTTP Error 404: Not Found
1010375
1010380
1010385
1010390
1010395
1010400
1010405
1010410
1010415
HTTP Error 404: Not Found
1010420
1010425
1010430
1010435
1010440
1010445
1010450
1010455
1010460
1010465
1010470
HTTP Error 404: Not Found
1010475
101048

1013495
1013500
1013505
1013510
1013515
1013520
HTTP Error 404: Not Found
1013525
1013530
1013535
1013540
1013545
1013550
1013555
1013560
1013565
1013570
HTTP Error 403: Forbidden
1013575
1013580
1013585
1013590
1013595
1013600
1013605
HTTP Error 404: Not Found
1013610
1013615
1013620
1013625
1013630
1013635
1013640
1013645
1013650
1013655
1013660
1013665
1013670
1013675
1013680
1013685
1013690
1013695
1013700
1013705
HTTP Error 404: Not Found
1013710
1013715
1013720
HTTP Error 404: Not Found
1013725
1013730
HTTP Error 404: Not Found
1013735
1013740
1013745
1013750
HTTP Error 404: Not Found
1013755
1013760
1013765
1013770
HTTP Error 404: Not Found
1013775
1013780
1013785
1013790
1013795
1013800
1013805
HTTP Error 404: Not Found
1013810
1013815
1013820
HTTP Error 404: Not Found
1013825
1013830
1013835
1013840
HTTP Error 404: Not Found
1013845
1013850
1013855
1013860
HTTP Error 404: Not Found
1013865
1013870
1013875
1013880
1013885
1013890
1013895
1013900
1013905
1013910
1013915
1013920


The read operation timed out
1016970
The read operation timed out
1016975
The read operation timed out
1016980
1016985
HTTP Error 404: Not Found
The read operation timed out
1016990
1016995
1017000
1017005
1017010
The read operation timed out
1017015
The read operation timed out
1017020
The read operation timed out
1017025
1017030
1017035
1017040
1017045
1017050
1017055
1017060
1017065
1017070
1017075
1017080
1017085
1017090
1017095
1017100
1017105
HTTP Error 404: Not Found
1017110
1017115
1017120
1017125
1017130
HTTP Error 404: Not Found
1017135
HTTP Error 404: Not Found
1017140
1017145
1017150
1017155
1017160
1017165
1017170
1017175
HTTP Error 404: Not Found
1017180
1017185
1017190
1017195
1017200
HTTP Error 503: Service Unavailable
1017205
1017210
1017215
1017220
1017225
1017230
1017235
1017240
1017245
1017250
1017255
The read operation timed out
The read operation timed out
1017260
The read operation timed out
1017265
1017270
The read operation timed out
1017275
1017280
1017285
101

1019775
1019780
1019785
HTTP Error 404: Not Found
1019790
1019795
1019800
HTTP Error 404: Not Found
1019805
1019810
1019815
1019820
1019825
1019830
1019835
1019840
1019845
1019850
1019855
1019860
1019865
1019870
1019875
1019880
1019885
1019890
1019895
HTTP Error 404: Not Found
1019900
HTTP Error 404: Not Found
1019905
1019910
1019915
1019920
1019925
1019930
1019935
1019940
1019945
HTTP Error 404: Not Found
1019950
1019955
1019960
1019965
1019970
1019975
1019980
1019985
HTTP Error 404: Not Found
1019990
1019995
1020000
1020005
1020010
1020015
1020020
1020025
1020030
1020035
HTTP Error 404: Not Found
1020040
HTTP Error 404: Not Found
1020045
1020050
1020055
1020060
1020065
HTTP Error 404: Not Found
1020070
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1020075
1020080
1020085
1020090
1020095
1020100
1020105
1020110
HTTP Error 404: Not Found
1020115
1020120
1020125
1020130
1020135
1020140
1020145
1020150
1020155
1020160
1020165
1020170
1020175
HTTP Error 404: Not Found
1020180
102018

HTTP Error 404: Not Found
1023270
1023275
1023280
HTTP Error 404: Not Found
1023285
1023290
1023295
1023300
1023305
1023310
1023315
1023320
1023325
1023330
HTTP Error 404: Not Found
1023335
1023340
1023345
1023350
1023355
1023360
1023365
HTTP Error 404: Not Found
1023370
1023375
1023380
1023385
HTTP Error 404: Not Found
1023390
1023395
HTTP Error 404: Not Found
1023400
'NoneType' object is not subscriptable
1023405
HTTP Error 404: Not Found
1023410
1023415
HTTP Error 404: Not Found
1023420
1023425
1023430
1023435
1023440
1023445
HTTP Error 404: Not Found
1023450
1023455
1023460
1023465
HTTP Error 404: Not Found
1023470
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1023475
1023480
HTTP Error 404: Not Found
1023485
1023490
1023495
1023500
1023505
1023510
1023515
1023520
1023525
1023530
1023535
1023540
1023545
1023550
1023555
1023560
HTTP Error 403: Forbidden
1023565
1023570
1023575
1023580
1023585
1023590
1023595
1023600
1023605
1023610
1023615
1023620
1023625
1023630
HTTP Error 40

...

1034015
1034020
1034025
1034030
1034035
1034040
1034045
1034050
1034055
1034060
1034065
1034070
1034075
1034080
HTTP Error 404: Not Found
1034085
1034090
1034095
1034100
1034105
1034110
1034115
1034120
1034125
HTTP Error 404: Not Found
1034130
1034135
1034140
HTTP Error 404: Not Found
1034145
1034150
1034155
1034160
'NoneType' object is not subscriptable
1034165
1034170
HTTP Error 403: Forbidden
HTTP Error 404: Not Found
1034175
HTTP Error 404: Not Found
1034180
1034185
1034190
1034195
1034200
1034205
1034210
1034215
1034220
1034225
1034230
1034235
1034240
1034245
1034250
1034255
HTTP Error 404: Not Found
1034260
1034265
HTTP Error 404: Not Found
1034270
1034275
1034280
1034285
1034290
1034295
1034300
1034305
1034310
1034315
1034320
1034325
1034330
1034335
1034340
1034345
1034350
1034355
1034360
1034365
HTTP Error 404: Not Found
1034370
1034375
1034380
HTTP Error 404: Not Found
1034385
HTTP Error 404: Not Found
1034390
1034395
HTTP Error 404: Not Found
1034400
1034405
1034410
1034415
1

1037660
1037665
1037670
HTTP Error 404: Not Found
1037675
1037680
1037685
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1037690
1037695
1037700
HTTP Error 404: Not Found
1037705
1037710
1037715
HTTP Error 404: Not Found
1037720
1037725
1037730
1037735
1037740
1037745
1037750
1037755
1037760
1037765
HTTP Error 404: Not Found
1037770
1037775
1037780
1037785
1037790
1037795
1037800
1037805
1037810
HTTP Error 404: Not Found
1037815
1037820
HTTP Error 404: Not Found
1037825
1037830
HTTP Error 404: Not Found
1037835
1037840
1037845
1037850
1037855
1037860
1037865
1037870
1037875
HTTP Error 404: Not Found
1037880
1037885
1037890
1037895
HTTP Error 404: Not Found
1037900
1037905
1037910
HTTP Error 404: Not Found
HTTP Error 403: Forbidden
1037915
1037920
1037925
1037930
1037935
1037940
1037945
1037950
1037955
1037960
1037965
1037970
1037975
HTTP Error 404: Not Found
1037980
1037985
1037990
1037995
1038000
1038005
1038010
1038015
1038020
1038025
1038030
1038035
HTTP Error 404: Not Found
10

1041390
1041395
1041400
1041405
1041410
1041415
1041420
1041425
'NoneType' object is not subscriptable
1041430
1041435
1041440
1041445
1041450
1041455
1041460
1041465
1041470
1041475
1041480
1041485
1041490
1041495
1041500
1041505
1041510
1041515
1041520
1041525
1041530
1041535
1041540
1041545
1041550
1041555
1041560
1041565
1041570
1041575
1041580
1041585
1041590
1041595
1041600
1041605
1041610
1041615
1041620
1041625
1041630
1041635
1041640
1041645
1041650
1041655
1041660
HTTP Error 403: Forbidden
1041665
1041670
1041675
1041680
1041685
1041690
1041695
1041700
1041705
1041710
1041715
1041720
1041725
1041730
1041735
1041740
1041745
1041750
1041755
1041760
1041765
1041770
1041775
1041780
1041785
1041790
1041795
HTTP Error 404: Not Found
1041800
1041805
1041810
1041815
1041820
1041825
1041830
1041835
1041840
1041845
1041850
1041855
1041860
1041865
1041870
1041875
1041880
1041885
1041890
1041895
1041900
1041905
1041910
1041915
1041920
1041925
1041930
1041935
1041940
HTTP Error 404: Not F

1045140
1045145
1045150
1045155
HTTP Error 404: Not Found
1045160
1045165
HTTP Error 404: Not Found
1045170
HTTP Error 404: Not Found
1045175
HTTP Error 404: Not Found
1045180
1045185
1045190
1045195
1045200
1045205
1045210
1045215
1045220
1045225
1045230
1045235
1045240
1045245
1045250
1045255
1045260
1045265
1045270
1045275
1045280
1045285
1045290
HTTP Error 404: Not Found
1045295
1045300
1045305
1045310
HTTP Error 404: Not Found
1045315
1045320
1045325
1045330
1045335
1045340
1045345
1045350
HTTP Error 403: Forbidden
1045355
1045360
1045365
1045370
1045375
1045380
1045385
1045390
1045395
1045400
HTTP Error 404: Not Found
1045405
1045410
1045415
1045420
HTTP Error 404: Not Found
1045425
1045430
HTTP Error 404: Not Found
1045435
1045440
1045445
1045450
1045455
1045460
1045465
1045470
1045475
1045480
HTTP Error 404: Not Found
1045485
1045490
1045495
HTTP Error 404: Not Found
1045500
1045505
1045510
1045515
1045520
1045525
1045530
1045535
1045540
1045545
1045550
1045555
1045560
1045565


1048530
HTTP Error 404: Not Found
1048535
1048540
1048545
1048550
1048555
1048560
1048565
1048570
HTTP Error 404: Not Found
1048575
1048580
HTTP Error 404: Not Found
1048585
1048590
1048595
1048600
1048605
1048610
1048615
HTTP Error 404: Not Found
HTTP Error 404: Not Found
1048620
1048625
1048630
1048635
1048640
1048645
1048650
1048655
1048660
1048665
1048670
1048675
1048680
1048685
1048690
1048695
HTTP Error 404: Not Found
1048700
1048705
1048710
1048715
1048720
1048725
1048730
1048735
1048740
1048745
1048750
1048755
1048760
1048765
1048770
1048775
1048780
1048785
1048790
1048795
1048800
1048805
1048810
1048815
1048820
1048825
1048830
1048835
1048840
1048845
1048850
1048855
HTTP Error 404: Not Found
1048860
1048865
1048870
1048875
1048880
1048885
1048890
1048895
1048900
1048905
1048910
1048915
1048920
1048925
1048930
1048935
1048940
The read operation timed out
1048945
1048950
1048955
1048960
1048965
HTTP Error 404: Not Found
1048970
1048975
1048980
1048985
1048990
1048995
1049000
104

In [56]:
#data = json.load(open('songData.json'))
#print(data['19'])

{'url': 'https://genius.com/Camron-losing-weight-pt-2-lyrics', 'title': 'Losing Weight, Pt. 2', 'artist': "Cam'ron", 'release_date': '2002-05-14', 'lyrics': 'b\'\\n\\n[Chorus: Cam\\\'ron]\\nAyo, fuck losing weight\\nI\\\'m back on these highways moving cakes\\nLife\\\'s based upon what I\\\'ma do today\\nCop a car, new estate\\nNa, fuck it get the beef and brocs: blue and gray\\nBaby due today\\nI got to move an eighth\\nFuck the scrutiny\\nY\\\'all niggas screwing me\\nKilla never let the drama slide\\nY\\\'all gone hear a nigga momma die\\nYell out "homicide"\\n\\n[Verse 1: Cam\\\'ron]\\nYo, 18 months? Please, that ain\\\'t facing time\\nI\\\'m stressed anyway, need it for vacation time\\nI\\\'ma do the right thing though, take shock anyway\\n6 months, right back on the damn block anyway\\nBut look, money from across the street\\nThink it\\\'s sweet, think he get money across the street\\nMe and my peeps often meet\\nAnd 5-0 they work for us, walk the beat\\nWalk with heat \\\'cause 

### Scrape song lyrics ###

## Python wrapper ##
You may need to run this code from the Terminal after cloning the repo
https://github.com/johnwmillr/geniusapi

In [10]:
# Create an instance of the API interface
import lyricsgenius as genius
api = genius.Genius(client_access_token)

In [11]:
# Search for an artist
artist = api.search_artist('Andy Shauf', max_songs=5)
print(artist)

Searching for Andy Shauf...

Song 1: "Alexander All Alone"
Song 2: "Begin Again"
Song 3: "Comfortable With Silence"
Song 4: "Covered in Dust"
Song 5: "Crushes"

Reached user-specified song limit (5).
Found 5 songs.
Done.
Andy Shauf, 5 songs


In [12]:
# Search for a specific song
song = api.search_song('Wendell Walker', artist.name)
artist.add_song(song)
print(artist)
print(artist.songs[0].lyrics)

Searching for "Wendell Walker"...
Done.
Andy Shauf, 6 songs
Alexander all alone
Smoking a cigarette
The last pack he’d ever buy
At least that’s what he said
He stood up to stretch his back
And fell down to the ground

Alexander all alone
'Till the neighbour caught a glimpse
Cried out for his wife
To call the ambulance
Alexander all alone
Felt them check his pulse
He heard them pronounce him dead

Hell is found inside of me
And nothing else will set me free
If hell is found inside of me
Then open me up and spill me out

Alexander wondered why
No life flashed before his eyes
Why his soul did not depart
Why he found no peace of mind
Would it take a little while?
Was it the same for everyone?
Alexander realized

That hell is found inside of me
And nothing else will set me free
If hell is found inside of me
Then open me up and spill me out
